# PISA Data Exploration
### by Andreja Ho



## Table of Contents
- [Introduction](#intro)
- [Preliminary Wrangling](#wrangling)
- [Univariate Exploration](#univariate)
- [Bivariate Exploration](#bivariate)
- [Multivariate Exploration](#multivariate)
- [Conclusion](#conclusion)

<a id='intro'></a>
## Introduction
Add text

<a id='wrangling'></a>
## Preliminary Wrangling

> Briefly introduce your dataset here.

> This document explores a dataset containing PISA's 2012 data. PISA is a survey of students' skills and knowledge as they approach the end of compulsory education. It is not a conventional school test. Rather than examining how well students have learned the school curriculum, it looks at how well prepared they are for life beyond school. Around 510,000 students in 65 economies globally took part in the PISA 2012 assessment of reading, mathematics, and science representing about 28 million 15-year-olds globally.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [ ]:
# Read in original dataset and explore what features are most suitable for the analysis
df_origin = pd.read_csv('pisa2012.csv',sep=',', encoding='latin-1',error_bad_lines=False, index_col=False, dtype='unicode')
pd.set_option('display.max_columns', None)
df_origin.head(3)

In [ ]:
# Dataset shape
df_origin.shape

> Load in your dataset and describe its properties through the questions below.
Try and motivate your exploration goals through this section.
Here describe the whole dataset and steps what did you do to get a clean dataset.

PISA 2012 dataset is very complex and the original dataset contains 485490 rows and 636 columns. For this analysis, I’ve chosen only a few columns of interest. To understand the variables (column headers), I used PISA’s data dictionary[link] and PISA’s codebook[link]. Both documents were necessary to choose variables of interest and how to properly interpret them. 

First, I looked through the dictionary to learn what data was gathered and define the idea of the interest. Next, I read through the codebook to decide what features or variables are best suited for the analysis. When I choose the columns of interest I used Excel `textjoin()` function in order to get all columns’ names in a list, separated with a comma and quotation marks.
 
I am interested in how specific features impact students’ performance on math, reading, and science testing. The features if interests are:
    - wealth,<br> 
    - cultural and home possessions,<br>
    - educational resources<br>
    - sense of belongings<br>
    - parents’ and siblings’ presence<br>
    - the educational level of parents<br><br>
Besides those features, I will be analyzing if there is any difference between<br>
    - gender<br>
    - schools that joined OECD<br>
    - country<br><br>
Additionally, I would like to research if<br> 
    - mathematics anxiety<br>
is correlated with performance in math and science.



In [2]:
# Read in csv file:
# - select only columns of interest (col_list)
#           - to get all columns names in a list, separated with comma with quotation marks `textjoin()` function was used
# - issues with readind the large csv:
#           - this why code was used:pd.read_csv('pisa2012.csv',sep=',', encoding='latin-1',error_bad_lines=False, index_col=False, dtype='unicode')

col_list = ['CNT','OECD','SCHOOLID','ST04Q01','ST09Q01','ST115Q01','ST11Q01','ST11Q02','ST11Q03','ST11Q04','ANXMAT','CULTPOS','FISCED','HEDRES','HOMEPOS','MISCED','WEALTH','ANCBELONG','PV1MATH','PV1READ','PV1SCIE']
df_short = pd.read_csv('pisa2012.csv',sep=',', encoding='latin-1',error_bad_lines=False, index_col=False, dtype='unicode', usecols=col_list)

In [3]:
# Display all columns for better view and display first five columns
pd.set_option('display.max_columns', None)
df_short.head(5)

,CNT,OECD,SCHOOLID,ST04Q01,ST09Q01,ST115Q01,ST11Q01,ST11Q02,ST11Q03,ST11Q04,ANXMAT,CULTPOS,FISCED,HEDRES,HOMEPOS,MISCED,WEALTH,ANCBELONG,PV1MATH,PV1READ,PV1SCIE
0,Albania,Non-OECD,0000001,Female,None,1,Yes,Yes,Yes,Yes,0.32,-0.48,"ISCED 3A, ISCED 4",-1.29,-2.61,"ISCED 3A, ISCED 4",-2.92,-0.7351,406.8469,249.5762,341.7009
1,Albania,Non-OECD,0000001,Female,None,1,Yes,Yes,NaN,Yes,NaN,1.27,"ISCED 3A, ISCED 4",1.12,1.41,"ISCED 5A, 6",0.69,NaN,486.1427,406.2936,548.9929
2,Albania,Non-OECD,0000001,Female,None,1,Yes,Yes,No,Yes,NaN,1.27,"ISCED 5A, 6",-0.69,0.14,"ISCED 5A, 6",-0.23,NaN,533.2684,401.21,499.6643
3,Albania,Non-OECD,0000001,Female,None,1,Yes,Yes,No,Yes,0.31,1.27,"ISCED 5A, 6",0.04,-0.73,"ISCED 3B, C",-1.17,NaN,412.2215,547.363,438.6796
4,Albania,Non-OECD,0000001,Female,None,2,Yes,Yes,Yes,NaN,1.02,1.27,"ISCED 3A, ISCED 4",-0.69,-0.57,None,-1.17,0.8675,381.9209,311.7707,361.5628


<a id='clean'></a>
## Clean

In [4]:
# Create a copy of original data sets
df = df_short.copy()

In [5]:
# Check the shape
df.shape

(485490, 21)

In [6]:
# Rename colum names for better understanding
# Columns were qiockly renamed with help of excel with `textjoin` and `concatenate` function
df.rename(columns={'CNT':'Country_code', 'OECD':'OECD_country', 'SCHOOLID':'School_ID', 'ST04Q01':'Gender', 'ST09Q01':'Truancy_Skip_whole_school_day', 'ST115Q01':'Truancy_Skip_classes_within_school_day', 'ST11Q01':'At_Home_Mother', 'ST11Q02':'At_Home_Father', 'ST11Q03':'At_Home_Brothers', 'ST11Q04':'At_Home_Sisters', 'ANXMAT':'Mathematics_Anxiety', 'CULTPOS':'Cultural_Possessions', 'FISCED':'Educational_level_of_father_ISCED', 'HEDRES':'Home_educational_resources', 'HOMEPOS':'Home_Possessions', 'MISCED':'Educational_level_of_mother_ISCED', 'WEALTH':'Wealth', 'ANCBELONG':'Sense_of_Belonging_to_School_Anchored', 'PV1MATH':'Plausible_value_mathematics', 'PV1READ':'Plausible_value_reading', 'PV1SCIE':'Plausible_value_science'}, inplace=True)

In [7]:
# Check renamed columns
df.head(3)

,Country_code,OECD_country,School_ID,Gender,Truancy_Skip_whole_school_day,Truancy_Skip_classes_within_school_day,At_Home_Mother,At_Home_Father,At_Home_Brothers,At_Home_Sisters,Mathematics_Anxiety,Cultural_Possessions,Educational_level_of_father_ISCED,Home_educational_resources,Home_Possessions,Educational_level_of_mother_ISCED,Wealth,Sense_of_Belonging_to_School_Anchored,Plausible_value_mathematics,Plausible_value_reading,Plausible_value_science
0,Albania,Non-OECD,0000001,Female,None,1,Yes,Yes,Yes,Yes,0.32,-0.48,"ISCED 3A, ISCED 4",-1.29,-2.61,"ISCED 3A, ISCED 4",-2.92,-0.7351,406.8469,249.5762,341.7009
1,Albania,Non-OECD,0000001,Female,None,1,Yes,Yes,NaN,Yes,NaN,1.27,"ISCED 3A, ISCED 4",1.12,1.41,"ISCED 5A, 6",0.69,NaN,486.1427,406.2936,548.9929
2,Albania,Non-OECD,0000001,Female,None,1,Yes,Yes,No,Yes,NaN,1.27,"ISCED 5A, 6",-0.69,0.14,"ISCED 5A, 6",-0.23,NaN,533.2684,401.21,499.6643


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485490 entries, 0 to 485489
Data columns (total 21 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   Country_code                            485490 non-null  object
 1   OECD_country                            485490 non-null  object
 2   School_ID                               485490 non-null  object
 3   Gender                                  485490 non-null  object
 4   Truancy_Skip_whole_school_day           479131 non-null  object
 5   Truancy_Skip_classes_within_school_day  479269 non-null  object
 6   At_Home_Mother                          460559 non-null  object
 7   At_Home_Father                          441036 non-null  object
 8   At_Home_Brothers                        400076 non-null  object
 9   At_Home_Sisters                         390768 non-null  object
 10  Mathematics_Anxiety                     314764 non-null 

Based on this [wikipedia](https://en.wikipedia.org/wiki/International_Standard_Classification_of_Education) site I have changed educational level of for better understanding:

| Pisa 2012 | From Wikipedia - ISCED 2011 | encoding used in this dataset
| :- | :- | :- |
|ISCED 1           | Primary education | ISCED 1
|ISCED 2           | Lower secondary education|  ISCED 2
|ISCED 3B, C       | Upper secondary education  | ISCED 3
|ISCED 3A, ISCED 4 | Post-secondary non-tertiary education   | ISCED 4
|ISCED 5B          | Short-cycle tertiary education | ISCED 5
|ISCED 5A, 6       | Bachelor's or equivalent  | ISCED 6
|None| No data| No data

In [9]:
# Rename education level - father ()
df.Educational_level_of_father_ISCED.value_counts()

ISCED 3A, ISCED 4    118890
ISCED 5A, 6          113406
ISCED 2               66728
ISCED 5B              61617
ISCED 3B, C           39789
ISCED 1               35938
None                  16535
Name: Educational_level_of_father_ISCED, dtype: int64

In [10]:
# Rename education level - mother ()
df.Educational_level_of_mother_ISCED.value_counts()

ISCED 3A, ISCED 4    126768
ISCED 5A, 6          114452
ISCED 5B              68219
ISCED 2               66650
ISCED 1               36556
ISCED 3B, C           35672
None                  18768
Name: Educational_level_of_mother_ISCED, dtype: int64

In [11]:
# Rename Truancy_Skip_whole_school_day
df.Truancy_Skip_whole_school_day.value_counts()

None                     385998
One or two times          75969
Three or four times       10882
Five or more times         6282
Name: Truancy_Skip_whole_school_day, dtype: int64

In [12]:
# Replace educational values with with easy understand values (see comments for explanation)
df["Educational_level_of_father_ISCED"] \
.replace({"ISCED 1": "ISCED 1",
          "ISCED 2": "ISCED 2",
          "ISCED 3B, C" : "ISCED 3",
          "ISCED 3A, ISCED 4": "ISCED 4",
          "ISCED 5B": "ISCED 5",
          "ISCED 5A, 6" :"ISCED 6",
         "None" : "No data"}, inplace=True)

In [13]:
# Replace educational values with easy understand values (see comments for explanation)
df["Educational_level_of_mother_ISCED"] \
.replace({"ISCED 1": "ISCED 1",
          "ISCED 2": "ISCED 2",
          "ISCED 3B, C" : "ISCED 3",
          "ISCED 3A, ISCED 4": "ISCED 4",
          "ISCED 5B": "ISCED 5",
          "ISCED 5A, 6" :"ISCED 6",
         "None" : "No data"}, inplace=True)

In [14]:
# Replace Truancy_Skip_whole_school_day values with with easy understand values
df["Truancy_Skip_whole_school_day"] \
.replace({"Five or more times": "5+ times",
          "Three or four times": "3-4 times",
          "One or two times" : "1-2 times",
          "None": "0 times"}, inplace=True)

***Rename function doesn't work on df"Truancy_Skip_whole_school_day" column?? cannot convert to different text and cannot convert to categorical datatype?***

In [15]:
# Check for correct transformation 
df.Educational_level_of_mother_ISCED.value_counts()

ISCED 4    126768
ISCED 6    114452
ISCED 5     68219
ISCED 2     66650
ISCED 1     36556
ISCED 3     35672
No data     18768
Name: Educational_level_of_mother_ISCED, dtype: int64

In [17]:
# Check for correct transformation - rename function didn't work??
df.Truancy_Skip_whole_school_day.value_counts()

None                     385998
One or two times          75969
Three or four times       10882
Five or more times         6282
Name: Truancy_Skip_whole_school_day, dtype: int64

In [18]:
pd.set_option('display.max_rows', None)
df.Country_code.value_counts().sort_index(ascending=False)

Vietnam                      4959
Uruguay                      5315
United States of America     4978
United Kingdom              12659
United Arab Emirates        11500
Turkey                       4848
Tunisia                      4407
Thailand                     6606
Switzerland                 11229
Sweden                       4736
Spain                       25313
Slovenia                     5911
Slovak Republic              4678
Singapore                    5546
Serbia                       4684
Russian Federation           5231
Romania                      5074
Qatar                       10966
Portugal                     5722
Poland                       4607
Peru                         6035
Perm(Russian Federation)     1761
Norway                       4686
New Zealand                  4291
Netherlands                  4460
Montenegro                   4744
Mexico                      33806
Massachusetts (USA)          1723
Malaysia                     5197
Macao-China   

In [19]:
# Combine names that have been broken down by states or cities with one unique name of the country
df = df.replace(['Connecticut (USA)', 'Florida (USA)', 'Massachusetts (USA)', 'United States of America'], 'USA')
df = df.replace(['Chinese Taipei'], 'Taiwan')
df = df.replace(['China-Shanghai', 'Hong Kong-China', 'Macao-China'], 'China')
df = df.replace(['Russian Federation', 'Perm(Russian Federation)'], 'Russia')

In [20]:
# Check for the right transformation
#df.Country_code.value_counts().sort_index(ascending=False)

<a id='datatypes'></a>
## Datatypes
It is important to change datatypes into appropriate datatypes for several reasons
- faster performance
- lower usage of the memory
- etc (add)

In [21]:
# Check datatype and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485490 entries, 0 to 485489
Data columns (total 21 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   Country_code                            485490 non-null  object
 1   OECD_country                            485490 non-null  object
 2   School_ID                               485490 non-null  object
 3   Gender                                  485490 non-null  object
 4   Truancy_Skip_whole_school_day           479131 non-null  object
 5   Truancy_Skip_classes_within_school_day  479269 non-null  object
 6   At_Home_Mother                          460559 non-null  object
 7   At_Home_Father                          441036 non-null  object
 8   At_Home_Brothers                        400076 non-null  object
 9   At_Home_Sisters                         390768 non-null  object
 10  Mathematics_Anxiety                     314764 non-null 

In [22]:
df.isnull().sum()

Country_code                                   0
OECD_country                                   0
School_ID                                      0
Gender                                         0
Truancy_Skip_whole_school_day               6359
Truancy_Skip_classes_within_school_day      6221
At_Home_Mother                             24931
At_Home_Father                             44454
At_Home_Brothers                           85414
At_Home_Sisters                            94722
Mathematics_Anxiety                       170726
Cultural_Possessions                       14133
Educational_level_of_father_ISCED          32587
Home_educational_resources                  7718
Home_Possessions                            5683
Educational_level_of_mother_ISCED          18405
Wealth                                      5893
Sense_of_Belonging_to_School_Anchored     177850
Plausible_value_mathematics                    0
Plausible_value_reading                        0
Plausible_value_scie

***Sense_of_Belonging_to_School_Anchored (177850) and  Mathematics_Anxiety (170726) has too many null values. For the this analysis purposes I have to drop null values (one of the reasons is to change datatype). For this reason I won't include these two features into my analysis.***

In [23]:
# Drop columns Sense_of_Belonging_to_School_Anchored and Mathematics_Anxiety
df.drop(['Sense_of_Belonging_to_School_Anchored', 'Mathematics_Anxiety'], axis=1, inplace=True)

In [24]:
# Drop null values
df.dropna(inplace=True)

In [25]:
df.shape

(325497, 19)

In [26]:
# Change Educational_level_of_father_ISCED to categort datatype
educational_classes = ['ISCED 1', 'ISCED 2', 'ISCED 3', 'ISCED 4', 'ISCED 5', 'ISCED 6', 'No data']
vclasses = pd.api.types.CategoricalDtype(ordered = True, categories = educational_classes)
df['Educational_level_of_father_ISCED'] = df['Educational_level_of_father_ISCED'].astype(vclasses)

In [27]:
# Change Educational_level_of_father_ISCED to categort datatype
educational_classes = ['ISCED 1', 'ISCED 2', 'ISCED 3', 'ISCED 4', 'ISCED 5', 'ISCED 6', 'No data']
vclasses = pd.api.types.CategoricalDtype(ordered = True, categories = educational_classes)
df['Educational_level_of_mother_ISCED'] = df['Educational_level_of_mother_ISCED'].astype(vclasses)

In [ ]:
# # Change Truancy_Skip_whole_school_day to categort datatype
# skip_school_day = ['None', '1-2 times', '3-4 times', '5+ times']
# vclassess = pd.api.types.CategoricalDtype(ordered = True, categories = skip_school_day)
# df['Truancy_Skip_whole_school_day'] = df['Truancy_Skip_whole_school_day'].astype(vclassess)

***Cannot convert to categorical datatype??***

In [30]:
# Change OECD_country to boolean
oecd_country = {'OECD': True, 'Non-OECD': False}
df["OECD_country"].replace(oecd_country, inplace=True)

In [31]:
# Chenge mother, father, brothers, sisters at home to boolean
home = {'Yes': True, 'No': False}
df["At_Home_Mother"].replace(home, inplace=True)
df["At_Home_Father"].replace(home, inplace=True)
df["At_Home_Brothers"].replace(home, inplace=True)
df["At_Home_Sisters"].replace(home, inplace=True)

In [32]:
# using dictionary to convert specific columns
convert_dict = {'Truancy_Skip_classes_within_school_day': int,
                'Cultural_Possessions' : float,
                'Home_educational_resources': float,
                'Home_Possessions': float,
                'Wealth': float,
                'Plausible_value_mathematics' : float,
                'Plausible_value_reading': float,
                'Plausible_value_science' : float,      
               }
  
df = df.astype(convert_dict)

In [33]:
# Check for datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325497 entries, 0 to 485489
Data columns (total 19 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   Country_code                            325497 non-null  object  
 1   OECD_country                            325497 non-null  bool    
 2   School_ID                               325497 non-null  object  
 3   Gender                                  325497 non-null  object  
 4   Truancy_Skip_whole_school_day           325497 non-null  object  
 5   Truancy_Skip_classes_within_school_day  325497 non-null  int32   
 6   At_Home_Mother                          325497 non-null  bool    
 7   At_Home_Father                          325497 non-null  bool    
 8   At_Home_Brothers                        325497 non-null  bool    
 9   At_Home_Sisters                         325497 non-null  bool    
 10  Cultural_Possessions            

<a id='store'></a>
### Store Data

In [34]:
# Export final dataframe
df.to_csv("Data/clean_PISA_df.csv", index=False)

<a id='features'></a>
### What is the structure of your dataset?

> Your answer here!

### What is/are the main feature(s) of interest in your dataset?

> Your answer here!

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> Your answer here!

<a id='univariate'></a>
## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

<a id='bivariate'></a>
## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

<a id='multivariate'></a>
## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!

<a id='conclusion'></a>
## Conclusion